# Azure AI Foundry - Gpt-image-1 webapp

Gpt-image-1 includes significant improvements on image generation: it has generate high-quality images in challenging scenarios and solving challenging prompts.

Currently the model has great zero-shot capabilities in image in-painting,
photorealistic photos, and wireframe designs.

> https://ai.azure.com/catalog/models/gpt-image-1

<img src="capture1.jpg">
<img src="capture2.jpg">
<img src="capture3.jpg">
<img src="capture4.jpg">

In [1]:
import base64
import datetime
import gradio as gr
import ipyplot
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
import sys
import tempfile

from dotenv import load_dotenv
from io import BytesIO
from openai import AzureOpenAI
from pathlib import Path
from PIL import Image
from rembg import remove

## 1. Settings

In [2]:
print(f"Today is {datetime.datetime.now().strftime('%Y-%m-%d')}")

Today is 2025-07-31


In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [4]:
load_dotenv("azure.env")

API_VERSION = "2025-04-01-preview"
MODEL = "gpt-image-1"  # As deployed in Azure AI Foundry

In [5]:
IMAGES_DIR = "images"

## 2. Helper

In [6]:
class GPTImageApp:
    def __init__(self):
        self.setup_directories()

    def setup_directories(self):
        """Create necessary directories for storing results"""
        self.results_dirs = {
            'edition': f'{IMAGES_DIR}/edition',
            'composition': f'{IMAGES_DIR}/composition',
            'inpainting': f'{IMAGES_DIR}/inpainting',
            'generation': f'{IMAGES_DIR}/generation',
        }

        for dir_path in self.results_dirs.values():
            os.makedirs(dir_path, exist_ok=True)

    def save_image_with_timestamp(self, image, directory, prefix):
        """Save image with timestamp and return the path"""
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]
        filename = f"azure_ai_foundry_gptimage1_{prefix}_{now}.jpg"
        filepath = os.path.join(directory, filename)

        if isinstance(image, str):  # base64 encoded
            img = Image.open(BytesIO(base64.b64decode(image)))
        else:
            img = image

        img.save(filepath)
        return filepath

    def image_generation(self,
                         prompt,
                         num_images=3,
                         size="1024x1024",
                         quality="high"):
        """Generate images using a text prompt"""
        try:
            client = AzureOpenAI(
                azure_endpoint=os.getenv("endpoint"),
                api_key=os.getenv("key"),
                api_version=API_VERSION,
            )

            params = {
                "model": MODEL,
                "prompt": prompt,
                "n": num_images,  # Number of images up to 10
                "quality": quality,  # low, medium, high
                "size": size,  # Values are: 1024x1024, 1536x1024, 1024x1536
                "output_compression": 70,  # JPEG compression level (0-100)
                "output_format": "jpeg",  # png, jpeg or webp
                "moderation":"auto",  # auto or low
                "background": "auto"  # auto, transparent or opaque

            }

            result = client.images.generate(**params)
            images_data = result.model_dump().get("data", [])
            encoded_images = [
                img.get("b64_json") for img in images_data if "b64_json" in img
            ]

            output_images = []
            for encoded_image in encoded_images:
                filepath = self.save_image_with_timestamp(
                    encoded_image, self.results_dirs['generation'],
                    "generated")
                output_images.append(filepath)

            return output_images, f"Successfully generated {len(output_images)} image(s)"

        except Exception as e:
            return [], f"Error generating images: {str(e)}"

    def image_edit(self,
                   image_file,
                   prompt,
                   num_images=3,
                   size="1024x1024",
                   quality="high"):
        """Edit an existing image based on text prompt"""
        try:
            # Save uploaded image temporarily
            with tempfile.NamedTemporaryFile(delete=False,
                                             suffix='.jpg') as tmp_file:
                image_file.save(tmp_file.name)
                temp_path = tmp_file.name

            headers = {"api-key": os.getenv("key")}
            aoai_name = re.search(r'https://(.*?)/openai',
                                  os.getenv("endpoint")).group(1)
            url = f"https://{aoai_name}/openai/deployments/{MODEL}/images/edits?api-version={API_VERSION}"

            files = {"image": open(temp_path, "rb")}

            data = {
                "prompt": prompt,
                "n": num_images,  # Number of images up to 10
                "quality": quality,  # low, medium, high
                "size": size,  # Values are: 1024x1024, 1536x1024, 1024x1536
                "output_compression": 70,  # JPEG compression level (0-100)
                "output_format": "jpeg",  # png, jpeg or webp
                "moderation": "auto",  # auto or low
                "background": "auto"  # auto, transparent or opaque
            }

            response = requests.post(url,
                                     headers=headers,
                                     files=files,
                                     data=data)
            response.raise_for_status()

            images_data = response.json()["data"]
            encoded_images = [img["b64_json"] for img in images_data]

            output_images = []
            for encoded_image in encoded_images:
                filepath = self.save_image_with_timestamp(
                    encoded_image, self.results_dirs['edition'], "edited")
                output_images.append(filepath)

            # Cleanup
            os.unlink(temp_path)
            files["image"].close()

            return output_images, f"Successfully edited image with {len(output_images)} result(s)"

        except Exception as e:
            return [], f"Error editing image: {str(e)}"

    def image_compose(self,
                      source_images,
                      prompt,
                      num_images=3,
                      size="1024x1024",
                      quality="high"):
        """Compose a new image using multiple source images"""
        try:
            if not source_images or len(source_images) == 0:
                return [], "Please upload at least one source image"

            # Save uploaded images temporarily
            temp_paths = []
            for img in source_images:
                with tempfile.NamedTemporaryFile(delete=False,
                                                 suffix='.jpg') as tmp_file:
                    img.save(tmp_file.name)
                    temp_paths.append(tmp_file.name)

            files = [(f"image[]", open(path, "rb")) for path in temp_paths]
            headers = {"api-key": os.getenv("key")}
            aoai_name = re.search(r'https://(.*?)/openai',
                                  os.getenv("endpoint")).group(1)
            url = f"https://{aoai_name}/openai/deployments/{MODEL}/images/edits?api-version={API_VERSION}"

            data = {
                "prompt": prompt,
                "n": num_images,  # Number of images up to 10
                "quality": quality,  # low, medium, high
                "size": size,  # Values are: 1024x1024, 1536x1024, 1024x1536
                "output_compression": 70,  # JPEG compression level (0-100)
                "output_format": "jpeg",  # png, jpeg or webp
                "moderation": "auto",  # auto or low
                "background": "auto"  # auto, transparent or opaque
            }

            response = requests.post(url,
                                     headers=headers,
                                     files=files,
                                     data=data)
            response.raise_for_status()

            images_data = response.json()["data"]
            encoded_images = [img["b64_json"] for img in images_data]

            output_images = []
            for encoded_image in encoded_images:
                filepath = self.save_image_with_timestamp(
                    encoded_image, self.results_dirs['composition'],
                    "composed")
                output_images.append(filepath)

            # Cleanup
            for file_tuple in files:
                file_tuple[1].close()
            for path in temp_paths:
                os.unlink(path)

            return output_images, f"Successfully composed {len(output_images)} image(s) from {len(source_images)} source images"

        except Exception as e:
            return [], f"Error composing images: {str(e)}"

    def image_inpainting(self,
                         initial_image,
                         prompt,
                         num_images=3,
                         size="1024x1024",
                         quality="high"):
        """Perform inpainting on an image by removing automatically the background of the image"""
        try:
            # Save uploaded image temporarily
            with tempfile.NamedTemporaryFile(delete=False,
                                             suffix='.jpg') as tmp_file:
                initial_image.save(tmp_file.name)
                temp_original = tmp_file.name

            # Create mask by removing background
            temp_mask = tempfile.NamedTemporaryFile(delete=False,
                                                    suffix='.jpg').name

            with open(temp_original, 'rb') as input_file:
                with open(temp_mask, 'wb') as output_file:
                    input_data = input_file.read()
                    output_data = remove(input_data)
                    output_file.write(output_data)

            headers = {"api-key": os.getenv("key")}
            aoai_name = re.search(r'https://(.*?)/openai',
                                  os.getenv("endpoint")).group(1)
            url = f"https://{aoai_name}/openai/deployments/{MODEL}/images/edits?api-version={API_VERSION}"

            files = {
                "image": open(temp_original, "rb"),
                "mask": open(temp_mask, "rb"),
            }
            data = {
                "prompt": prompt,
                "n": num_images,  # Number of images up to 10
                "quality": quality,  # low, medium, high
                "size": size,  # Values are: 1024x1024, 1536x1024, 1024x1536
                "output_compression": 70,  # JPEG compression level (0-100)
                "output_format": "jpeg",  # png, jpeg or webp
                "moderation": "auto",  # auto or low
                "background": "auto"  # auto, transparent or opaque
            }

            response = requests.post(url,
                                     headers=headers,
                                     files=files,
                                     data=data)
            response.raise_for_status()

            images_data = response.json()["data"]
            encoded_images = [img["b64_json"] for img in images_data]

            output_images = []
            for encoded_image in encoded_images:
                filepath = self.save_image_with_timestamp(
                    encoded_image, self.results_dirs['inpainting'],
                    "inpainted")
                output_images.append(filepath)

            # Cleanup
            files["image"].close()
            files["mask"].close()
            os.unlink(temp_original)
            os.unlink(temp_mask)

            return output_images, f"Successfully inpainted image with {len(output_images)} result(s)"

        except Exception as e:
            return [], f"Error inpainting image: {str(e)}"


## 3. Gradio webapp

In [7]:
# Initialize the app
app = GPTImageApp()


def create_interface():
    with gr.Blocks(title="Azure AI Foundry GPT-Image-1 Studio",
                   theme=gr.themes.Soft()) as webapp:

        gr.Markdown("""
        # 🎨 Azure AI Foundry GPT-Image-1 Studio
        
        A comprehensive web application for AI-powered image generation, editing, composition, and inpainting using Azure AI Foundry GPT-Image-1 model.
        
        **Features:**
        - 🖼️ **Image Generation**: Create images from text prompts
        - ✏️ **Image Editing**: Modify existing images with text instructions
        - 🎭 **Image Composition**: Combine multiple images into one
        - 🎨 **Image Inpainting**: Fill in or modify parts of images (background replacement)
        """)

        with gr.Tabs():
            # Image Generation Tab
            with gr.Tab("🖼️ 1. Image Generation"):
                gr.Markdown("### Generate images from text descriptions")

                # Example prompts for generation
                gen_examples = [
                    "Generate a high-quality, realistic image of a stylish and cozy modern apartment interior. Include large windows with natural light, minimalist furniture, warm neutral tones, and a clean inviting atmosphere. The space should feel both elegant and livable.",
                    "A man with a dog in Paris street close to a red car with a licence plate named 'a red car'",
                    "A beautiful house in tropical modernism style inside of a forest and full of trees and plants",
                    "A 3D cartoon-style action figure of a man named John, displayed inside a clear plastic blister package like a collectible toy. The figure resembles a Caucasian male with black hair, wearing a blue polo with the French flag in the upper left of the polo, black fitted pants, and black dress shoes. He has a titanium wedding band on his left hand. The packaging has a clean, modern, cartoonish design. The top of the packaging is bright blue, with bold white text that reads: 'John' as the name, and below it, in smaller text: 'I love Azure AI'. Inside the blister package, next to the figure, are miniature job-related accessories: a black iPhone, black Bose over-ear headphones, a tall coffee cup with the French flag, and a silver Microsoft Surface laptop. The overall look is colorful, professional, and playful — like a limited-edition tech professional collectible figure sold in stores."
                    "A futuristic cityscape at sunset with flying cars and neon lights",
                    "A cozy coffee shop interior with warm lighting and vintage furniture",
                    "A high-resolution, commercial-quality perfume bottle with elegant design and transparent background"
                ]

                with gr.Row():
                    with gr.Column(scale=2):
                        gen_prompt = gr.Textbox(
                            label="✏️ Prompt",
                            placeholder="Describe the image you want to generate...",
                            lines=3)

                        # Dropdown for example prompts
                        gen_example_dropdown = gr.Dropdown(
                            choices=gen_examples,
                            label="💡 Examples prompts (click to use)",
                            value=None,
                            interactive=True)

                        with gr.Row():
                            gen_num_images = gr.Slider(
                                1,
                                10,
                                value=3,
                                step=1,
                                label="Number of images")
                            gen_size = gr.Dropdown(choices=["1024x1024", "1536x1024", "1024x1536"],
                                                   value="1024x1024",
                                                   label="Image resolution")
                            gen_quality = gr.Dropdown(
                                choices=["high", "medium", "low"],
                                value="high",
                                label="Image quality")

                        gen_button = gr.Button("🎨 IMAGE GENERATION",
                                               variant="primary")

                    with gr.Column(scale=4):
                        gen_output = gr.Gallery(label="🖼️ Image resolution",
                                                columns=2,
                                                rows=2)
                        gen_status = gr.Textbox(label="✅ Status",
                                                interactive=False)

                # Event handlers
                gen_example_dropdown.change(lambda x: x if x else "",
                                            inputs=[gen_example_dropdown],
                                            outputs=[gen_prompt])

                gen_button.click(
                    app.image_generation,
                    inputs=[gen_prompt, gen_num_images, gen_size, gen_quality],
                    outputs=[gen_output, gen_status])

            # Image Editing Tab
            with gr.Tab("✏️ 2. Image editing"):
                gr.Markdown("### Edit existing images with text instructions")

                # Example prompts for editing
                edit_examples = [
                    "Change the color of the walls in light blue",
                    "Add a darker sky",
                    "Add some beard to the man",
                    "Add a hat to the man",
                    "Add sunglasses to the face",
                    "Add some people in the background",
                ]

                with gr.Row():
                    with gr.Column(scale=2):
                        edit_image = gr.Image(label="📤 Upload image to edit",
                                              type="pil")
                        edit_prompt = gr.Textbox(
                            label="✏️ Edit instructions",
                            placeholder="Describe how you want to modify the image...",
                            lines=3)

                        # Dropdown for example prompts
                        edit_example_dropdown = gr.Dropdown(
                            choices=edit_examples,
                            label="💡 Examples edit instructions (click to use)",
                            value=None,
                            interactive=True)

                        with gr.Row():
                            edit_num_images = gr.Slider(
                                1,
                                10,
                                value=3,
                                step=1,
                                label="Number of images")
                            edit_size = gr.Dropdown(choices=["1024x1024", "1536x1024", "1024x1536"],
                                                    value="1024x1024",
                                                    label="Image resolution")
                            edit_quality = gr.Dropdown(
                                choices=["high", "medium", "low"],
                                value="high",
                                label="Image quality")

                        edit_button = gr.Button("✏️ IMAGE EDITION",
                                                variant="primary")

                    with gr.Column(scale=4):
                        edit_output = gr.Gallery(label="🖼️ Edited images",
                                                 columns=2,
                                                 rows=2)
                        edit_status = gr.Textbox(label="✅ Status",
                                                 interactive=False)

                # Event handlers
                edit_example_dropdown.change(lambda x: x if x else "",
                                             inputs=[edit_example_dropdown],
                                             outputs=[edit_prompt])

                edit_button.click(app.image_edit,
                                  inputs=[
                                      edit_image, edit_prompt, edit_num_images,
                                      edit_size, edit_quality
                                  ],
                                  outputs=[edit_output, edit_status])

            # Image Composition Tab
            with gr.Tab("🎭 3. Image composition"):
                gr.Markdown(
                    "### Combine multiple images into a single composition")

                # Example prompts for composition
                compose_examples = [
                    "Combine the different images into a single one",
                    "Generate a photorealistic image of a man wearing these items in the reference pictures. The man should walk in a busy street. We must see the Eiffel tower in the background.",
                    "Show a person wearing these items in a professional office setting",
                    "Display the items as a coordinated outfit in a fashion magazine layout"
                ]

                with gr.Row():
                    with gr.Column(scale=2):
                        compose_images = gr.File(
                            label="📤 Upload source image(s)",
                            file_count="multiple",
                            file_types=["image"])
                        compose_prompt = gr.Textbox(
                            label="✏️ Composition instructions",
                            placeholder="Describe how you want to combine the uploaded images...",
                            lines=3)

                        # Dropdown for example prompts
                        compose_example_dropdown = gr.Dropdown(
                            choices=compose_examples,
                            label="💡 Example composition instructions (click to use)",
                            value=None,
                            interactive=True)

                        with gr.Row():
                            compose_num_images = gr.Slider(
                                1,
                                10,
                                value=3,
                                step=1,
                                label="Number of images")
                            compose_size = gr.Dropdown(
                                choices=["1024x1024", "1536x1024", "1024x1536"],
                                value="1024x1024",
                                label="Image resolution")
                            compose_quality = gr.Dropdown(
                                choices=["high", "medium", "low"],
                                value="high",
                                label="Image quality")

                        compose_button = gr.Button("🎭 IMAGE COMPOSITION",
                                                   variant="primary")

                    with gr.Column(scale=4):
                        compose_output = gr.Gallery(label="🖼️ Composed images",
                                                    columns=2,
                                                    rows=2)
                        compose_status = gr.Textbox(label="✅ Status",
                                                    interactive=False)

                def handle_compose(files, prompt, num_images, size, quality):
                    if not files:
                        return [], "Please upload at least one image"

                    # Convert uploaded files to PIL Images
                    images = []
                    for file in files:
                        try:
                            img = Image.open(file.name)
                            images.append(img)
                        except Exception as e:
                            return [], f"Error loading image {file.name}: {str(e)}"

                    return app.image_compose(images, prompt, num_images, size,
                                             quality)

                compose_button.click(handle_compose,
                                     inputs=[
                                         compose_images, compose_prompt,
                                         compose_num_images, compose_size,
                                         compose_quality
                                     ],
                                     outputs=[compose_output, compose_status])

            # Image Inpainting Tab
            with gr.Tab("🎨 4. Image inpainting"):
                gr.Markdown("### Fill in or modify parts of images using inpainting technique")

                # Example prompts for inpainting
                inpaint_examples = [
                    "Add a busy street in the background. We must see the Eiffel tower in the background",
                    "The car should be close to the sea. Change the color of the car to green. Change the licence plate to 'I am a green car'",
                    "Change the color car to blue, white and red. Add a transparent background.",
                    "Add a beautiful sunset sky with dramatic clouds",
                    "Replace the background with a tropical beach scene",
                ]

                with gr.Row():
                    with gr.Column(scale=2):
                        inpaint_image = gr.Image(
                            label="📤 Upload image for inpainting", type="pil")
                        inpaint_prompt = gr.Textbox(
                            label="✏️ Inpainting instructions",
                            placeholder="Describe what to add or change in the background...",
                            lines=3)

                        # Dropdown for example prompts
                        inpaint_example_dropdown = gr.Dropdown(
                            choices=inpaint_examples,
                            label="💡 Example inpainting instructions (click to use)",
                            value=None,
                            interactive=True)

                        with gr.Row():
                            inpaint_num_images = gr.Slider(
                                1,
                                10,
                                value=3,
                                step=1,
                                label="Number of images")
                            inpaint_size = gr.Dropdown(
                                choices=["1024x1024", "1536x1024", "1024x1536"],
                                value="1024x1024",
                                label="Image resolution")
                            inpaint_quality = gr.Dropdown(
                                choices=["high", "medium", "low"],
                                value="high",
                                label="Image quality")

                        inpaint_button = gr.Button("🎨 IMAGE INPAINTING",
                                                   variant="primary")

                    with gr.Column(scale=4):
                        inpaint_output = gr.Gallery(label="🖼️ Inpainted images",
                                                    columns=2,
                                                    rows=2)
                        inpaint_status = gr.Textbox(label="✅ Status",
                                                    interactive=False)

                # Event handlers
                inpaint_example_dropdown.change(
                    lambda x: x if x else "",
                    inputs=[inpaint_example_dropdown],
                    outputs=[inpaint_prompt])

                inpaint_button.click(app.image_inpainting,
                                     inputs=[
                                         inpaint_image, inpaint_prompt,
                                         inpaint_num_images, inpaint_size,
                                         inpaint_quality
                                     ],
                                     outputs=[inpaint_output, inpaint_status])

        gr.Markdown("""
        ---
                        
        ### 🎯 Tips:
        - Be specific and detailed in your prompts for better results
        - Try different quality settings for various use cases
        - Experiment with different image sizes based on your needs

        All results powered by Azure AI Foundry
        """)

    return webapp

In [8]:
webapp = create_interface()

webapp.launch(
    share=True,
    show_error=True,
    favicon_path=None,  # Add your favicon path here
    auth=None,  # Add authentication if needed: ("username", "password")
    max_threads=40, # By default, Gradio uses a thread pool (inherited from FastAPI) with a maximum of 40 threads.
    debug=False,
)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8d09b9ee2b4bf8d4e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 4. All generated images

In [12]:
gen_image_files_list = []

for root, dirs, files in os.walk(IMAGES_DIR):
    for file in files:
        if file.lower().endswith((".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp")):
            gen_image_files_list.append(os.path.join(root, file))


gen_image_files_list

['images/composition/azure_ai_foundry_gptimage1_composed_20250731_085845_086.jpg',
 'images/composition/azure_ai_foundry_gptimage1_composed_20250731_085845_179.jpg',
 'images/composition/azure_ai_foundry_gptimage1_composed_20250731_085845_235.jpg',
 'images/edition/azure_ai_foundry_gptimage1_edited_20250731_085704_576.jpg',
 'images/edition/azure_ai_foundry_gptimage1_edited_20250731_085704_655.jpg',
 'images/edition/azure_ai_foundry_gptimage1_edited_20250731_085704_706.jpg',
 'images/generation/azure_ai_foundry_gptimage1_generated_20250731_085520_839.jpg',
 'images/generation/azure_ai_foundry_gptimage1_generated_20250731_085520_915.jpg',
 'images/generation/azure_ai_foundry_gptimage1_generated_20250731_085520_966.jpg',
 'images/inpainting/azure_ai_foundry_gptimage1_inpainted_20250731_090015_624.jpg',
 'images/inpainting/azure_ai_foundry_gptimage1_inpainted_20250731_090015_706.jpg',
 'images/inpainting/azure_ai_foundry_gptimage1_inpainted_20250731_090015_758.jpg']

In [13]:
if len(gen_image_files_list) > 0:
    ipyplot.plot_images(gen_image_files_list, img_width=200)